In [5]:
from train import init_experiment, get_decoder, get_vocab
import os
import torch
import torch.nn as nn
import dataset

params = init_experiment('clip-rnn-en')


lang = params['lang']
batch_size = params['batch_size']
embed_size = params['embed_size']
hidden_size = params['hidden_size']
num_layers = params['num_layers']
lr = params['lr']
clip_model = params['clip_model']
model_dir = params['model_dir']
num_epochs = params['num_epochs']
print_every = params['print_every']
save_every = params['save_every']
model_name = params['model_name']
checkpoint = 'checkpoints/rnn-eng-hd512-6-.ckpt'
use_checkpoint = False

os.makedirs( model_dir , exist_ok=True)
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vocab_size = get_vocab(lang)
loss_function = nn.CrossEntropyLoss()

encoder, data_loader_train = dataset.get_loader_f8k(lang, batch_size, clip_model=clip_model, device=device)
encoder.eval()    
decoder, start_epoch = get_decoder(embed_size, hidden_size, vocab_size, num_layers, checkpoint=checkpoint, use_checkpoint=use_checkpoint, device=device)
decoder.to(device)
decoder.eval()

{
    "batch_size": 24,
    "checkpoint": "",
    "clip_model": "ViT-B/16",
    "embed_size": 512,
    "hidden_size": 512,
    "lang": "en",
    "lr": 0.01,
    "model_dir": "./checkpoints",
    "model_name": "rnn-eng-hd512",
    "num_epochs": 60,
    "num_layers": 1,
    "print_every": 10,
    "save_every": 1,
    "use_checkpoint": false
}


DecoderRNN(
  (word_embedding): Embedding(8922, 512)
  (lstm): LSTM(512, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=8922, bias=True)
)

In [6]:
loaded_state_dict = torch.load(checkpoint)
new_state_dict = decoder.state_dict()

In [7]:
loaded_state_dict.keys()

odict_keys(['positional_embedding', 'text_projection', 'logit_scale', 'visual.class_embedding', 'visual.positional_embedding', 'visual.proj', 'visual.conv1.weight', 'visual.ln_pre.weight', 'visual.ln_pre.bias', 'visual.transformer.resblocks.0.attn.in_proj_weight', 'visual.transformer.resblocks.0.attn.in_proj_bias', 'visual.transformer.resblocks.0.attn.out_proj.weight', 'visual.transformer.resblocks.0.attn.out_proj.bias', 'visual.transformer.resblocks.0.ln_1.weight', 'visual.transformer.resblocks.0.ln_1.bias', 'visual.transformer.resblocks.0.mlp.c_fc.weight', 'visual.transformer.resblocks.0.mlp.c_fc.bias', 'visual.transformer.resblocks.0.mlp.c_proj.weight', 'visual.transformer.resblocks.0.mlp.c_proj.bias', 'visual.transformer.resblocks.0.ln_2.weight', 'visual.transformer.resblocks.0.ln_2.bias', 'visual.transformer.resblocks.1.attn.in_proj_weight', 'visual.transformer.resblocks.1.attn.in_proj_bias', 'visual.transformer.resblocks.1.attn.out_proj.weight', 'visual.transformer.resblocks.1.at